Fake News Classifier Using  Bi directional LSTM  
Dataset:- https://www.kaggle.com/c/fake-news/data

In [1]:
import pandas as pd


In [2]:
#It is done encoding problem
df=pd.read_csv("train.csv",error_bad_lines=False,engine="python")

<ipython-input-2-999b3a976366>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("train.csv",error_bad_lines=False,engine="python")
Skipping line 597: unexpected end of data


In [3]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
590,590,"""Authoritarianism"": How the West demonizes str...",Jafe Arnoldski (noreply@blogger.com),"November 22, 2016 - Deena Stryker, Katehon - ...",1
591,591,Swedish Journo: Sweden Will Collapse Without I...,Chris Tomlinson,Swedish writer Åsa Linderborg has claimed the ...,0
592,592,Is your promising internet career over now Vin...,newsbiscuit editorial team,,1
593,593,"In Iowa, Trump Voters Are Unfazed by Controver...",Trip Gabriel,"MONTICELLO, Iowa — The Table of Knowledge w...",0


In [4]:
df.head()#If 1 then it is fake news classifier and if 0 not a fake news

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(595, 5)

In [ ]:
# we are going to take our title and train it (lstm)

In [6]:
#check whether we have any null data set or not and add them all
df.isnull().sum() # null value is present

id         0
title     14
author    69
text       2
label      0
dtype: int64

In [7]:
# now in this case what should we do
# 1 - drop the values or
# 2 - replace the values in a better way
# Since it is text data , so we cannot replace the value
# so we will drop the null values
df=df.dropna()

In [8]:
df.shape #see the shape change

(512, 5)

In [9]:
# we going to work with the title because of lstm
# all the independent features
x=df.drop('label',axis=1) # drop label column of the x axis

In [10]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
590,590,"""Authoritarianism"": How the West demonizes str...",Jafe Arnoldski (noreply@blogger.com),"November 22, 2016 - Deena Stryker, Katehon - ..."
591,591,Swedish Journo: Sweden Will Collapse Without I...,Chris Tomlinson,Swedish writer Åsa Linderborg has claimed the ...
592,592,Is your promising internet career over now Vin...,newsbiscuit editorial team,
593,593,"In Iowa, Trump Voters Are Unfazed by Controver...",Trip Gabriel,"MONTICELLO, Iowa — The Table of Knowledge w..."


In [11]:
## get dependent features
y=df['label']

In [12]:
y

0      1
1      0
2      1
3      1
4      1
      ..
590    1
591    0
592    1
593    0
594    1
Name: label, Length: 512, dtype: int64

In [13]:
x.shape

(512, 4)

In [14]:
y.shape

(512,)

In [15]:
import tensorflow as tf

In [17]:
# check whether the data set is balanced or not
y.value_counts()

0    298
1    214
Name: label, dtype: int64

In [16]:
tf.__version__

'2.13.0'

In [19]:
from tensorflow.keras.layers import Embedding # embeddinglayer - convert word into vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences # it create a padding layer
from tensorflow.keras.models import Sequential # sequential - A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
from tensorflow.keras.preprocessing.text import one_hot # one hot encodong
from tensorflow.keras.layers import LSTM #lstm
from tensorflow.keras.layers import Dense # dense -Just your regular densely-connected NN layer.
from tensorflow.keras.layers import Bidirectional # bidirectional lstm

In [20]:
### vocabulary size
voc_size=5000 #ohe will be big based on this
# now use the title column and use it

OHE Representation

In [21]:
messages=x.copy() # I will not work on x because it contain all the independent features

In [22]:
messages['title'][2] # 1 - It is the index position

'Why the Truth Might Get You Fired'

In [23]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
590,590,"""Authoritarianism"": How the West demonizes str...",Jafe Arnoldski (noreply@blogger.com),"November 22, 2016 - Deena Stryker, Katehon - ..."
591,591,Swedish Journo: Sweden Will Collapse Without I...,Chris Tomlinson,Swedish writer Åsa Linderborg has claimed the ...
592,592,Is your promising internet career over now Vin...,newsbiscuit editorial team,
593,593,"In Iowa, Trump Voters Are Unfazed by Controver...",Trip Gabriel,"MONTICELLO, Iowa — The Table of Knowledge w..."


In [24]:
# reset the index if incase there is a mismatched
messages.reset_index(inplace=True)

In [ ]:
# we have to apply ohe on the title

In [25]:
import nltk # put ?? to get the documentation
import re #re-reguler expression
from nltk.corpus import stopwords

In [26]:
# we are going to download all the stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# we are first going to do stopwords and lemmatization on the title

In [27]:
### Data Preprocessing
from nltk.stem.porter import PorterStemmer #stemming purpose
ps=PorterStemmer()
corpus=[] #below this all will be in the corpus
for i in range (0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ' , messages['title'][i]) # remove all the special character and replace it with space
  review=review.lower()
  review=review.split() # words by words

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' ' . join(review) # join all the review
  corpus.append(review) #combine all in the corpus
# review will have list of words
# and if the word is not in stopwords (english) we have to stem it
# if it is present in stopwords I have to ignore
#re.sub- returns all the string value

In [29]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
#we can apply lemmatization but it will take more time

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
# ohe
onehot_rep=[one_hot(words,voc_size)for words in corpus] #index will be captured
onehot_rep

[[3810, 2977, 336, 2329, 284, 4343, 2907, 3406, 3127, 741],
 [4735, 501, 1740, 1226, 3224, 2734, 89],
 [1118, 2160, 3696, 2533],
 [2908, 451, 2717, 4036, 2418, 2810],
 [4092, 3224, 4771, 3490, 3080, 534, 3224, 4716, 1411, 3811],
 [659,
  4822,
  1887,
  4988,
  1484,
  4061,
  2799,
  1351,
  1686,
  2585,
  4350,
  3032,
  537,
  4536,
  89],
 [1943, 2936, 2404, 3100, 4665, 3274, 506, 3093, 4089, 51, 4906],
 [2397, 2627, 569, 1367, 1260, 2498, 4061, 1269, 4089, 51, 4906],
 [1892, 148, 4650, 914, 4195, 714, 1551, 1340, 4061, 4282],
 [2826, 2294, 2596, 2374, 1720, 4942, 4451, 3871],
 [1249, 1394, 1773, 2407, 2682, 3471, 1115, 3067, 3741, 4210, 1150],
 [4036, 1548, 284, 714, 4061, 1260],
 [4464, 4483, 2157, 4036, 2316, 4165, 363, 94, 748],
 [3509, 474, 4116, 379, 1096, 850, 939, 4089, 51, 4906],
 [1372, 1875, 3300, 1393, 507, 4089, 51, 4906],
 [286, 2857, 1928, 1183, 3329, 1341, 1896, 4176, 1964, 1629],
 [2540, 3073, 501],
 [1763, 164, 933, 4072, 4061, 4520, 8, 89],
 [194, 903, 1740, 281

In [32]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [33]:
onehot_rep[1]

[4735, 501, 1740, 1226, 3224, 2734, 89]

## **Embedding Representation (basically it is word 2 vec)**

In [34]:
#padding
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3406 3127  741]
 [   0    0    0 ... 3224 2734   89]
 [   0    0    0 ... 2160 3696 2533]
 ...
 [   0    0    0 ...  956 3411  712]
 [   0    0    0 ... 4089   51 4906]
 [   0    0    0 ... 2238 4216 1336]]


In [35]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4735,  501, 1740, 1226, 3224, 2734,   89], dtype=int32)

In [38]:
from nltk.translate import metrics
## creating model
embedding_vector_features=40 # each word will get converted of vector 40 size(just play with any number-hyper parameter tuning)
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100))) # 100 neurons forward and 100 neuron backwards
model.add(Dense(1,activation='sigmoid')) # sigmoid-binary, Dense layer is 1
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary()) #embedding layer- 20 input, 40 output

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [39]:
len(embedded_docs),y.shape # first one is input and second one is output

(512, (512,))

In [40]:
import numpy as np
x_final=np.array(embedded_docs) #all the embedded form i see in the form of list(vectors) convert it into array
y_final=np.array(y)

In [42]:
x_final.shape,y_final.shape # (512, 20)- 512 input, 20- padding

((512, 20), (512,))

**Model Training**

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( x_final, y_final, test_size=0.33, random_state=42)

In [44]:
### Finally model training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
6/6 [==============================] - 6s 397ms/step - loss: 0.6874 - accuracy: 0.5743 - val_loss: 0.6747 - val_accuracy: 0.5976
Epoch 2/10
6/6 [==============================] - 1s 92ms/step - loss: 0.6786 - accuracy: 0.5743 - val_loss: 0.6639 - val_accuracy: 0.5976
Epoch 3/10
6/6 [==============================] - 0s 67ms/step - loss: 0.6595 - accuracy: 0.5743 - val_loss: 0.6525 - val_accuracy: 0.5976
Epoch 4/10
6/6 [==============================] - 0s 70ms/step - loss: 0.6351 - accuracy: 0.6239 - val_loss: 0.6305 - val_accuracy: 0.7101
Epoch 5/10
6/6 [==============================] - 0s 65ms/step - loss: 0.5797 - accuracy: 0.8805 - val_loss: 0.5796 - val_accuracy: 0.7811
Epoch 6/10
6/6 [==============================] - 0s 66ms/step - loss: 0.4472 - accuracy: 0.9388 - val_loss: 0.5016 - val_accuracy: 0.8047
Epoch 7/10
6/6 [==============================] - 0s 67ms/step - loss: 0.2703 - accuracy: 0.9184 - val_loss: 0.3898 - val_accuracy: 0.8817
Epoch 8/10
6/6 [==========

Dropout layer (improve the accuracy)

Performance Metrics and Accuracy

In [45]:
y_pred=model.predict(x_test)

6/6 [==============================] - 1s 9ms/step


In [52]:
y_pred

array([[0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
    

In [54]:
np.where(y_pred>=0.5,1,0)

array([[0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
    

In [55]:
#threshhold (limit)
y_pred=np.where(y_pred > 0.5,1,0) # when ever it is greater than 0 , i will take it as 1-fake and 0- real
#if i want to remove the thresh hold use (AUC ROC Curve)
#AUC ROC Curve - what is the probability you should select

In [56]:
y_pred

array([[0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
    

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
confusion_matrix(y_test,y_pred)

array([[94,  7],
       [21, 47]])

In [59]:
# accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.834319526627219

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.82      0.93      0.87       101
           1       0.87      0.69      0.77        68

    accuracy                           0.83       169
   macro avg       0.84      0.81      0.82       169
weighted avg       0.84      0.83      0.83       169



In [ ]:
# to increase accuracy use dropper,after applying run from model training (it will also reduce over fitting)
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])